In [1]:
import qube
import glob
import importlib
import inspect
import os
import ast
from collections import defaultdict
from sklearn.base import BaseEstimator
from git import Repo, Remote
from itertools import zip_longest
from qube.simulator.tracking.trackers import FixedRiskTrader
from qube.simulator.tracking.sizers import FixedSizer

%qubed

c:\Users\dmitr\devs\venvs\py39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


 >>> For support multiprocessing under Windows 'multiprocess' module should be installed
Qube> Cant find "projects/" folder for adding to python path !

              .+-------+ 
            .' :     .'|   QUBE | Quantitative Backtesting Environment
           +-------+'  |  
           |   : * |   |   (c) 2022,  ver. 0.3.7
           |  ,+---|---+ 
           |.'     | .'    
           +-------+'    


# Find all strategies in filesystem

In [2]:
def iter_funcs(class_node):
    decls = {}
    for n in ast.walk(class_node): 
        if isinstance(n, ast.FunctionDef) and n.name == '__init__':
            args = []
            for x in n.args.args:
                args.append(x.arg)
            for x, dx in zip_longest(reversed(args), reversed(n.args.defaults)):
                if x != 'self':
                    # print(f"{x} = {dx.value if dx else None}")
                    decls[x] = dx.value if dx and isinstance(dx, ast.Constant) else None
    # return dict(reversed(decls))
    return dict(reversed(list(decls.items())))

def flatten(decl_list):
    for d in decl_list:
        try:
            yield d.id
        except AttributeError:
            try:
                yield d.func.id
            except AttributeError:
                yield None

def is_decorated(decorator_list, decorator_name):
    for x in flatten(decorator_list):
        if x == decorator_name:
            return True
    return False

In [3]:
strats = defaultdict(list)
for file in glob.glob('**/*.py', recursive=True):
    name = os.path.splitext(os.path.basename(file))[0]
    # Ignore __ files
    if name.startswith("__"):
        continue

    with open(file, 'r') as f:
        src = f.read()

    try:
        class_node = ast.parse(src)
    except:
        continue

    nodes = [node for node in ast.walk(class_node) if isinstance(node, ast.ClassDef)]
    for n in nodes:
        if is_decorated(n.decorator_list, 'signal_generator'):
            strats[file].append((n.name, ast.get_docstring(n), iter_funcs(n)))

In [4]:
for f, sd in strats.items():
    strs = ""
    for sn, descr, pars in sd:
        descr = (': ' + descr.replace('\n', ' ').strip('" ')) if descr else ''
        strs += f"   |-- {sn} {descr} \n   |   {pars}\n   |\n"

    rst = f""" - {f} -
{strs}"""
    print(rst)
    # break

In [5]:
print(dict(strats))

{}

: 

In [37]:
file = 'incubator/research/meanreverting/nimble/nimblers.py'
with open(file, 'r') as f:
    src = f.read()
    try:
        class_node = ast.parse(src)
    except:
        raise

    for n in ast.walk(class_node): 
        if isinstance(n, ast.FunctionDef) and n.name == '__init__':
            args = []
            for x in n.args.args:
                args.append(x.arg)

            for x, dx in zip_longest(reversed(args), reversed(n.args.defaults)):
                if x != 'self':
                    print(f"{x} = {dx.value if dx and isinstance(dx, ast.Constant) else None}")

meanrevert = True
stop_loss_pct = 0
atr_avg = kama
atr_period = None
avg = tema
nstd = 1
period = 12
capital = None
timeframe = 1h
reverting = False
long_only = False
vol_factor = 1
vol_smoother = kama
vol_periods = 100
vol_timeframe = 1H
max_position_size = 0
max_capital_in_risk = 0
stop_loss_pct = 0
pvalue = 0.05
beta = 0.05
alpha = 0.9
capital = None
timeframe = 1h
revert = False
index_name = INDEX
fdi_period = None
long_only = False
vol_factor = 1
vol_smoother = kama
vol_periods = 100
vol_timeframe = 1H
max_position_size = 0
max_capital_in_risk = 0
stop_loss_pct = 0
pvalue = 0.05
beta = 0.05
alpha = 0.9
capital = None
no_regime_filter = False
no_fdi_filter = False
timeframe = 1h
revert = False
long_only = False
index_name = INDEX
filter_period = None
vol_ratio = 5
vol_factor = 1
vol_smoother = kama
vol_periods = 100
vol_timeframe = 1H
max_position_size = 0
max_capital_in_risk = 0
stop_loss_pct = 0
pvalue = 0.05
beta = 0.05
alpha = 0.9
capital = None
revert = False
timeframe = 1h
fi

In [38]:
from qube.booster.boosterai import Boo
Boo.fzf('Insaner').show()

>>> insane-trend-breakout.yml - incubator/data/experiments/insane/insane-trend-breakout.yml
 ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
 |-[0]	Insane-breakout-default                           	[Insane]	2020-04-06:now
 | 		 incubator.research.momentum.insane.insaners.Insaner1Risk
 | 		 INSANE Strategy - trend breakout | BTCUSDT | fixed capital | stop | take | 1H timeframe | number symbols 1

 |-[1]	Insane-breakout-opt-stage-1                       	[Insane]	2020-04-06:now
 | 		 incubator.research.momentum.insane.insaners.Insaner1Risk
 | 		 INSANE Strategy - trend breakout - opt stage 1 | BTCUSDT | fixed capital | stop | take | 1H timeframe | number symbols 1

 |-[2]	Insane-breakout-big-range                         	[Insane]	2020-04-06:now
 | 		 incubator.research.momentum.insane.insaners.Insaner1Risk
 | 		 INSANE Strategy - trend breakout - all option | BTCUSDT | fixed capital | stop | take | 1H timeframe | number symbols 1

>>>

	- actions: .run(<num>), .clean(<num>), .show(<num>), .load(<num>)

# Git control

In [7]:
r = Repo('Qube2exp')
git = r.git
r.head.commit.tree

<git.Tree "786e60650ee6f908b4fc9b7854475961ffadcf82">

In [8]:
r.head.commit.hexsha

'772487e8b58e20114dfbf07fcd0851bff964cea3'

In [9]:
[d.a_path for d  in r.index.diff(None)]

['experiments/booster/0. Ideas - reading filesystem.ipynb']

In [116]:
git.add('.')
git.commit(m="Second commit")

'[main d906932] Second commit\n 1 file changed, 280 insertions(+), 18 deletions(-)'

In [117]:
hsh = r.commit('main').binsha.hex()
hsh

'd906932f93efb0f65b4f2561cac0c7d33f787f42'

In [66]:
from qube.booster.boosterai import _generate_name_like_id
_generate_name_like_id(hsh, 3), _generate_name_like_id(hsh, 5, 5)

('Xoj', 'Gupuw')

# New simulator | live design

In [ ]:
class TimeData:
    def freq(self, timeframe: str) -> 'TimeData':
        return self

    def timeframe(self):
        return None

class Storage:
    """
    How to get data
    """
    def load(self, symbols: list) -> TimeData:
        return None

class TradeOp:

    def __add__(self, other: 'TradeOp') -> 'TradeOp':
        return self

class Market:
    def get_availabe_balance(self) -> float:
        return 0

class IStrategy:
    market: Market

    def train(self, data: TimeData) -> 'IStrategy':
        return self

    def signals(self, data: TimeData):
        return None
    
    def trade(self, side, where=None) -> TradeOp:
        pass

    def get_availabe_capital(self) -> float:
        return self.market.get_availabe_balance()
    
    def tracker(self):
        # portfolio / single ?
        return None

In [ ]:
from qube import ta
from qube import variate

class RsiTrader(IStrategy):

    def __init__(self, period=12) -> None:
        self.period = period

    def train(self):
        return self

    def signals(self, data: TimeData) -> TradeOp:
        r = ta.rsi(data, self.period)    # how to pass context (if needed at all) ?
        mp = data.low + data.high

        return self.trade(
            +1, where=(r[1] < 20) & (r > 20) & (mp > data.close), stop=data.low[1]
        ) + self.trade(
            -1, where=(r[1] > 80) & (r < 80) & (mp < data.close), stop=data.high[1]
        )
 
class RsiTrader2(IStrategy):

    def __init__(self, period=12) -> None:
        self.period = period

    def train(self):
        return self

    def signals(self, data: TimeData) -> TradeOp:
        r = ta.rsi(data, self.period)
        mp = data.low + data.high

        return self.trade(
            +1, where=(r[1] < 20) & (r > 20) & (mp > data.close), at=data.high[1], stop=data.low[1], gtc=1,
        ) + self.trade(
            -1, where=(r[1] > 80) & (r < 80) & (mp < data.close), at=data.low[1], stop=data.high[1], gtc=1,
        )


class PortfolioTrader(IStrategy):
    def __init__(self) -> None:

        

In [ ]:
data = Storage('mongo', 'binancef').load(['BTCUSDT', 'ETHUSDT', 'SOLUSDT', 'AVAXUSDT'])
data = Storage('kdb', 'binance').load(['BTCUSDT', 'ETHUSDT', 'SOLUSDT', 'AVAXUSDT'])

In [ ]:
test = Market.Test(
    'binance-um.vip0-usdt', 

    # data.freq('ticks'), 

    data[['BTC', 'ETH']].freq('ticks'), 

    start='2023-07-01', stop='2023-12-02', 
)

live = Market.Live(
    'binance-um.vip0-usdt', 
    account='a2323098976897',
    balance={'USDT': 45000},   # ????
    leverage=2, 
    commissions='vip0-usdt',
)

In [ ]:
r = test.run(
    variate(RsiTrader, period=[14, 28, 36]),

    capital=10000, leverage=1, mode='portfolio',
    train_period='4w', trade_period='1w',         # walk forward

    custom = {
        'BTCUSDT': dict(period=12)
    }
)

In [ ]:
r.tearsheet()
r.equity(drop=['ETH'])